In [1]:
df=spark.createDataFrame([["20-12-2020"]],schema="dt string")
df.show()

+----------+
 dt|
+----------+
20-12-2020|
+----------+

--------------------------------------------------------------------------- 
 TypeError Traceback (most recent call last)
 <command-2165085649096504> in <module> 
 ----> 1 df = spark . createDataFrame ( [ [ "10-12-2020" ] ] , schema = "dt date" ) 
 2 df . show ( ) 

 /databricks/spark/python/pyspark/sql/session.py in createDataFrame (self, data, schema, samplingRatio, verifySchema) 
 651 return super(SparkSession, self).createDataFrame(
 652 data, schema, samplingRatio, verifySchema)
 --> 653 return self . _create_dataframe ( data , schema , samplingRatio , verifySchema ) 
 654 
 655 def _create_dataframe ( self , data , schema , samplingRatio , verifySchema ) : 

 /databricks/spark/python/pyspark/sql/session.py in _create_dataframe (self, data, schema, samplingRatio, verifySchema) 
 683 rdd , schema = self . _createFromRDD ( data . map ( prepare ) , schema , samplingRatio ) 
 684 else : 
 --> 685 rdd , schema = self . _createFromLocal ( map ( prepare , data ) , schema ) 
 686 jrdd = self . _jvm . SerDeUtil . toJavaArray ( rdd . _to_java_object_rdd ( ) ) 
 687 jdf = self . _jsparkSession . applySchemaToPythonRDD ( jrdd . rdd ( ) , schema . json ( ) ) 

 /databricks/spark/python/pyspark/sql/session.py in _createFromLocal (self, data, schema) 
 468 write temp files . 
 469 """
 --> 470 data , schema = self . _wrap_data_schema ( data , schema ) 
 471 return self . _sc . parallelize ( data ) , schema
 472 

 /databricks/spark/python/pyspark/sql/session.py in _wrap_data_schema (self, data, schema) 
 444 # make sure data could consumed multiple times 
 445 if not isinstance ( data , list ) : 
 --> 446 data = list ( data ) 
 447 
 448 if schema is None or isinstance ( schema , ( list , tuple ) ) : 

 /databricks/spark/python/pyspark/sql/session.py in prepare (obj) 
 658 
 659 def prepare ( obj ) : 
 --> 660 verify_func ( obj ) 
 661 return obj
 662 elif isinstance ( schema , DataType ) : 

 /databricks/spark/python/pyspark/sql/types.py in verify (obj) 
 1406 def verify ( obj ) : 
 1407 if not verify_nullability ( obj ) : 
 -> 1408 verify_value ( obj ) 
 1409 
 1410 return verify

 /databricks/spark/python/pyspark/sql/types.py in verify_struct (obj) 
 1387 "length of fields (%d)" % (len(obj), len(verifiers))))
 1388 for v , ( _ , verifier ) in zip ( obj , verifiers ) : 
 -> 1389 verifier ( v ) 
 1390 elif hasattr ( obj , "__dict__" ) : 
 1391 d = obj . __dict__

 /databricks/spark/python/pyspark/sql/types.py in verify (obj) 
 1406 def verify ( obj ) : 
 1407 if not verify_nullability ( obj ) : 
 -> 1408 verify_value ( obj ) 
 1409 
 1410 return verify

 /databricks/spark/python/pyspark/sql/types.py in verify_default (obj) 
 1400 def verify_default ( obj ) : 
 1401 assert_acceptable_types ( obj ) 
 -> 1402 verify_acceptable_types ( obj ) 
 1403 
 1404 verify_value = verify_default

 /databricks/spark/python/pyspark/sql/types.py in verify_acceptable_types (obj) 
 1285 if type ( obj ) not in _acceptable_types [ _type ] : 
 1286 raise TypeError(new_msg("%s can not accept object %r in type %s"
 -> 1287 % (dataType, obj, type(obj))))
 1288 
 1289 if isinstance ( dataType , StringType ) : 

 TypeError : field dt: DateType can not accept object '10-12-2020' in type <class 'str'>

In [3]:
import pyspark.sql.functions as f

In [4]:
df_date=df.select(f.to_date("dt","dd-MM-YYYY ").alias("date_formatted"))

In [5]:
df_date.show()

+--------------+
date_formatted|
+--------------+
 2019-12-29|
+--------------+

In [6]:
df_converted=df_date.select(f.date_format("date_formatted","YYYY/MM/dd HH:mm:ss").alias("new_format"))

In [7]:
df_converted.show()

+-------------------+
 new_format|
+-------------------+
2020/12/29 00:00:00|
+-------------------+

In [9]:
df=spark.createDataFrame([["10/12/2020 10:20:20"]],schema="time string")

In [10]:
df_time=df.select(f.to_timestamp("time","dd/MM/YYYY HH:mm:ss").alias("timestamp_formatted"))

In [11]:
df_time.show()

+-------------------+
timestamp_formatted|
+-------------------+
2019-12-29 10:20:20|
+-------------------+

In [12]:
df_data=spark.createDataFrame([("2020-08-23",1),("2020-08-24",1),("2020-08-25",1)],schema="sold_date string,quantity int")

In [13]:
df_data_formatted=df_data.select(f.to_date("sold_date").alias("sold_date"),"quantity")

In [14]:
df_data_formatted.show()

+----------+--------+
 sold_date|quantity|
+----------+--------+
2020-08-23| 1|
2020-08-24| 1|
2020-08-25| 1|
+----------+--------+

In [15]:
df_data_formatted.select("*",f.trunc("sold_date",'week').alias("weekly_date")).show()

+----------+--------+-----------+
 sold_date|quantity|weekly_date|
+----------+--------+-----------+
2020-08-23| 1| 2020-08-17|
2020-08-24| 1| 2020-08-24|
2020-08-25| 1| 2020-08-24|
+----------+--------+-----------+

In [16]:
df_data_formatted.select("*",f.trunc("sold_date",'week').alias("weekly_date")).groupby(f.col("weekly_date")).sum("quantity").show()

+-----------+-------------+
weekly_date|sum(quantity)|
+-----------+-------------+
 2020-08-24| 2|
 2020-08-17| 1|
+-----------+-------------+

In [17]:
df_data_formatted.select("*",f.trunc("sold_date",'month').alias("weekly_date")).groupby(f.col("weekly_date")).sum("quantity").show()

+-----------+-------------+
weekly_date|sum(quantity)|
+-----------+-------------+
 2020-08-01| 3|
+-----------+-------------+